In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
import math 
from ast import literal_eval
import pickle
import json

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'beaming-opus-337709'
!gcloud config set project {project_id}
!gsutil ls

Are you sure you wish to set property [core/project] to beaming-opus-337709?

Do you want to continue (Y/n)?  Y

Updated property [core/project].
You are attempting to access protected data with no configured
credentials. Please visit https://cloud.google.com/console#/project
and sign up for an account, and then run the "gcloud auth login"
command to configure gsutil to use these credentials.


In [ ]:
bucket_name = 'ipcat_bucket2'

!gsutil -m cp -r gs://ipcat_bucket2/store_data_gh/results_scan_on_port_80/round_7/zipped/zgrab_result_instance-05_7_443_20210903.json.bz2  /content/drive/My\ Drive/Data/banner.json.bz2

AccessDeniedException: 403 sahanr@scorelab.org does not have storage.objects.list access to the Google Cloud Storage bucket.
CommandException: 1 file/object could not be transferred.


In [ ]:
!sudo apt-get install bzip2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bzip2 is already the newest version (1.0.6-8.1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!bzip2 -d ./Data/banner.json.bz2

In [ ]:
columnlist = [
       'ip',
       'data.http.result.response.headers.server',
       'data.http.result.response.status_code',
       'data.http.result.response.headers.connection',
       'data.http.result.response.headers.via',
       'data.http.result.response.headers.accept_ranges',
       'data.http.result.response.headers.cache_control',
       'data.http.result.response.headers.retry_after',
       'data.http.result.response.content_length',
       'data.http.result.response.headers.content_type',
       'data.http.result.response.headers.location',
       'data.http.result.response.headers.etag',
]


In [ ]:
def getgeneralCloudfront(x):
  if('(CloudFront)' in str(x)):
      return 'CloudFront'
  else:
      return x

def getgeneralServerNames(x):
  if('nginx' in str(x)):
      return 'nginx'
  elif('Apache' in str(x)):
      return 'Apache'
  else:
      return x

def getgeneralphp(x):
  if (pd.isna(x)):
    return -1
  else:
    if('PHP' in x):
      return 'PHP'

def changenullaszero(x):
  try:
      if (pd.isna(x)): 
        return -1
      else:
        return 1
  except:
     print('Error Occured')
  finally:
    return -1

def getCIDRBlockLength(ipadd):
   if(pd.isna(ipadd)):
     return 0
   else:
      if(len(ipadd.split(','))==1):
        return (2**(32-int(ipadd.split('/')[1])))
      else:
        sum=0
        for i in ipadd.split(','):
          sum=sum + (2**(32-int(i.split('/')[1])))
        return sum 

def arrayLiteral(stringarray):
  if(pd.isna(stringarray)):
     return 0
  return len(literal_eval(stringarray))

def removeSquareBrackets(text):
  if(pd.isna(text)):
     return -1
  return literal_eval(text)[0];

In [ ]:
with open('label_encodings','rb') as f:
  encoder_dict = pickle.load(f)

In [ ]:
encoder_dict['data.http.result.response.status_code'].classes_

array(['-1.0', '101.0', '200.0', '201.0', '204.0', '299.0', '301.0',
       '302.0', '303.0', '307.0', '308.0', '400.0', '401.0', '402.0',
       '403.0', '404.0', '405.0', '406.0', '407.0', '409.0', '410.0',
       '412.0', '418.0', '421.0', '422.0', '426.0', '470.0', '500.0',
       '501.0', '502.0', '503.0', '504.0', '508.0', '511.0', '523.0',
       '561.0', '596.0', '599.0', '801.0', '999.0'], dtype=object)

In [ ]:
def encodingcheck(val,column):
  column = encoder_dict[column].classes_
  return (val in column ) and  np.where( column ==val)[0][0] or -1

In [ ]:
#encodingcheck(str(float(500)),'data.http.result.response.status_code')

27

In [ ]:
finallist = []
finaltemplist = []
count = 0

with open('Data/banner.json') as file:
      for line in file:
          count=count+1
          row = json.loads(line.rstrip())
          df = pd.json_normalize(row)
          templist = []
          temptemplist = []
          for column in columnlist:
            if column in df.columns:
              (column == 'ip') and templist.append(df[column].values[0])
              (column == 'data.http.result.response.headers.server') and templist.append(encodingcheck(getgeneralServerNames(df[column].values[0]),column))
              (column == 'data.http.result.response.status_code') and templist.append(encodingcheck(str(float(df[column].values[0])),column))
              (column == 'data.http.result.response.headers.connection') and templist.append(encodingcheck(df[column].values[0][0],column))
              (column == 'data.http.result.response.headers.via') and templist.append(encodingcheck(changenullaszero(df[column].values[0]),column))
              (column == 'data.http.result.response.headers.accept_ranges') and templist.append(encodingcheck(df[column].values[0][0],column))
              (column == 'data.http.result.response.headers.cache_control') and templist.append(encodingcheck(df[column].values[0][0],column))
              (column == 'data.http.result.response.headers.retry_after') and templist.append(encodingcheck(df[column].values[0][0],column))
              (column == 'data.http.result.response.content_length') and templist.append(df[column].values[0])
              (column == 'data.http.result.response.headers.content_type') and templist.append(encodingcheck(df[column].values[0][0],column))
              (column == 'data.http.result.response.headers.location') and templist.append(encodingcheck(changenullaszero(df[column].values[0]),column))
              (column == 'data.http.result.response.headers.etag') and templist.append(encodingcheck(changenullaszero(df[column].values[0]),column))
              temptemplist.append(df[column].values[0])
              continue
            else:
              templist.append(-1)
              temptemplist.append(-1)
          finaltemplist.append(temptemplist)
          finallist.append(templist)
          if(count%5000==0):
            print(count)

In [ ]:
len(finallist[0])

12

In [ ]:
finallist[1000]

['159.69.32.133', 479, 6, 6, -1, -1, -1, -1, 162, 44, -1, -1]

In [ ]:
finallist  = pd.DataFrame(finallist, columns =columnlist)

In [ ]:
finaltemplist[127035]

['144.73.228.49', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [ ]:
finallist.iloc[127035]

ip                                                 144.73.228.49
data.http.result.response.headers.server                      -1
data.http.result.response.status_code                         -1
data.http.result.response.headers.connection                  -1
data.http.result.response.headers.via                         -1
data.http.result.response.headers.accept_ranges               -1
data.http.result.response.headers.cache_control               -1
data.http.result.response.headers.retry_after                 -1
data.http.result.response.content_length                      -1
data.http.result.response.headers.content_type                -1
data.http.result.response.headers.location                    -1
data.http.result.response.headers.etag                        -1
Name: 127035, dtype: object